# 基于计数方法的改进
## 点互信息
上一节中的，上下文单词出现的次数表示共现矩阵，这种”原始“的次数不具备好的性质。如果只看单词的出现次数，那么与 drive car 相比，the car出现的次数会更多，the 和 car 的相关性 drive 和 car 的相关性更强, 这不合理。

点互信息（Pointwise Mutual Information, PMI）, 定义如下：

<img width="300" src="images/2_4_1.png">

P（x）表示x发生的概率，P（y）表示y发生的概率，P（x, y）表示x和y同时发生的概率。PMI的值越高，表明相关性越强。

<img width="300" src="images/2_4_2.png">

如上图, C(x), C(y) 为 x, y 出现的次数， C(x, y)为x,y的共现次数， 语料库单词总数量为 N。

当两个词的共现次数为 0 时, PMI($\log_2^0=-\infty$) 为负无穷大，因此实践中都使用 **正的的点互信息（Positive PMI, PPMI）**

\begin{equation} PPMI(x, y) = max(0, PMI(x, y)) \end{equation}

> 注意，发生概率、出现次数，是在共现矩阵中的，如 "goodbye and i" 中，"and" 只出现了 1 次，但是在共现矩阵中的出现次数为 2。 


In [ ]:
import numpy as np


def ppmi(C, verbose=False, eps=1e-8):
    M = np.zeros_like(C, dtype=np.float32)
    # 所有元素之和
    N = np.sum(C)
    # 共现矩阵各行对应位置的元素相加，得行向量, 每个元素值表示索引（word_id）对应 word 的出现次数
    S = np.sum(C, axis=0)
    total = C.shape[0] * C.shape[1]  # 共现矩阵元素数量
    cnt = 0

    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            pmi = np.log2(C[i, j] * N / (S[i] * S[j]) + eps)
            M[i, j] = max(0, pmi)

            if verbose:
                cnt += 1
                if cnt % (total // 100 + 1) == 0:
                    print('%.1f%% done' % (100 * cnt / total))
    return M


def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')
    word_to_id = {}
    id_to_word = {}

    for word in words:
        if word not in word_to_id:
            word_id = len(word_to_id)
            word_to_id[word] = word_id
            id_to_word[word_id] = word
    corpus = np.array([word_to_id[w] for w in words])
    return corpus, word_to_id, id_to_word


def create_co_matrix(corpus, vocab_size, window_size=1):
    """ 创建共现矩阵 """
    corpus_size = len(corpus)
    # 以 0 初始化一个 vocab_size x vocab_size 的矩阵
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    # 遍历语料库 corpus 中的每一个 word id
    for idx, word_id in enumerate(corpus):
        # 根据窗口大小遍历上下文的所有单词
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id][left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id][right_word_id] += 1

    return co_matrix


text = 'You say goodbye adn I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)
W = ppmi(C)
np.set_printoptions(precision=3)  # 有效位数

print('covariance matrix')
print(C)
print('-' * 50)
print('PPMI')
print(W)

通过以上代码将 共现矩阵 转化为了一个更好的指标形成的 PPMI矩阵, 获得了更好的单词向量。 但是随着语料库词汇量的增加，矩阵也会无限增大，很难处理。 观察发现， PPMI矩阵中，有很多为0的元素，表示点互信息很低，重要性很低。 针对这些，可以降维，降低数据量、噪声、增加稳健性。

## 降维
减少不重要的向量维度，保留重要信息。

正交：

矩阵矩阵：

对角矩阵：